In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Folder paths for landmark data
data_dir = r"C:\Users\aniru\OneDrive\Desktop\fitfreak\data\landmarks"

# Map exercises to numerical labels
exercise_labels = {
    "barbell_row": 0,
    "dumbbell_biceps_curl": 1,
    "dumbbell_overhead_shoulder_press": 2,
    "pushups": 3,
    "side_lateral_raise": 4
}

# Define a fixed sequence length for training
MAX_SEQUENCE_LENGTH = 100  # Adjust based on video length and FPS

# Load CSV data from each folder
def load_data(data_dir):
    X, y = [], []
    for exercise, label in exercise_labels.items():
        exercise_dir = os.path.join(data_dir, exercise)
        for file in os.listdir(exercise_dir):
            if file.endswith(".csv"):
                df = pd.read_csv(os.path.join(exercise_dir, file))
                
                # Extract pose landmark data (ignore frame index)
                sequence = df.iloc[:, 1:].values  # Shape: (frames, num_features)

                # Pad or truncate sequences to maintain consistency
                sequence = pad_sequences([sequence], maxlen=MAX_SEQUENCE_LENGTH, padding="post", dtype="float32")[0]

                X.append(sequence)
                y.append(label)
    
    # Convert to NumPy arrays
    X = np.array(X, dtype="float32")
    y = to_categorical(y, num_classes=len(exercise_labels))  # One-hot encoding

    return X, y

# Load the data
X, y = load_data(data_dir)

# Print dataset shape
print(f"X shape: {X.shape}, y shape: {y.shape}")


X shape: (124, 100, 132), y shape: (124, 5)


In [5]:
from tensorflow.keras.layers import Input, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D

def build_transformer_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Positional Encoding
    positional_encoding = tf.expand_dims(tf.range(start=0, limit=input_shape[0], delta=1), axis=0)
    x = inputs + tf.keras.layers.Embedding(input_dim=input_shape[0], output_dim=input_shape[1])(positional_encoding)

    # Transformer Encoder Layers
    for _ in range(4):  # Number of Transformer Encoder layers
        x = MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
        x = LayerNormalization(epsilon=1e-6)(x)
        x = Dense(256, activation="relu")(x)

    # Classification Head
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

# Build and train the Transformer Model
transformer_model = build_transformer_model(input_shape, num_classes)
transformer_model.summary()

history = transformer_model.fit(X, y, validation_split=0.2, epochs=15, batch_size=16)

# Save model
transformer_model.save("models/exercise_form_transformer.h5")
print("Transformer model saved successfully!")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100, 132)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100, 132)  │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 132)  │    272,004 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 100, 132)  │        264 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100, 256)  │     34,048 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 256)  │    526,080 │ dense_2[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 256)  │        512 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100, 256)  │     65,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 256)  │    526,080 │ dense_3[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 256)  │        512 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 100, 256)  │     65,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 256)  │    526,080 │ dense_4[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 256)  │        512 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 100, 256)  │     65,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ dense_5[0][0]     │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 5)         │      1,285 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,084,753 (7.95 MB)

 Trainable params: 2,084,753 (7.95 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 222ms/step - accuracy: 0.2924 - loss: 2.7196 - val_accuracy: 0.0000e+00 - val_loss: 4.3849
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.3529 - loss: 1.8137 - val_accuracy: 0.0000e+00 - val_loss: 2.7481
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.3029 - loss: 1.6893 - val_accuracy: 0.0000e+00 - val_loss: 2.3131
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.2482 - loss: 1.3339 - val_accuracy: 0.0000e+00 - val_loss: 2.8756
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.3167 - loss: 1.3908 - val_accuracy: 0.0000e+00 - val_loss: 2.9643
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - accuracy: 0.2427 - loss: 1.4385 - val_accuracy: 0.0000e+00 - val_loss: 1.8956
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.3068 - loss: 1.3299 - val_accuracy: 0.0000e+00 - val_loss: 2.4769
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.3020 - loss: 1.2760 - val_

Transformer model saved successfully!


In [7]:
# Folder path for test landmarks
test_data_dir = r"C:\Users\aniru\OneDrive\Desktop\fitfreak\data\test_landmarks"

def load_test_data(test_data_dir):
    X_test, y_test = [], []
    for exercise, label in exercise_labels.items():
        exercise_dir = os.path.join(test_data_dir, exercise)
        for file in os.listdir(exercise_dir):
            if file.endswith(".csv"):
                df = pd.read_csv(os.path.join(exercise_dir, file))
                sequence = df.iloc[:, 1:].values

                # Pad sequences
                sequence = pad_sequences([sequence], maxlen=MAX_SEQUENCE_LENGTH, padding="post", dtype="float32")[0]
                
                X_test.append(sequence)
                y_test.append(label)

    X_test = np.array(X_test, dtype="float32")
    y_test = to_categorical(y_test, num_classes=len(exercise_labels))  # One-hot encode
    return X_test, y_test

# Load test dataset
X_test, y_test = load_test_data(test_data_dir)
print(f"Test Data Shape: X_test={X_test.shape}, y_test={y_test.shape}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\aniru\\OneDrive\\Desktop\\fitfreak\\data\\test_landmarks\\dumbbell_biceps_curl'